In [14]:
#Agregamos las bibliotecas que vayamos a usar
from google.cloud import bigquery
client = bigquery.Client()

In [15]:
#Se escogieron estos datos por
#
#
#
#
#Se agrega la referencia al conjunto de datos y se recogen
dataset_ref = client.dataset("covid19_public_forecasts", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)
